In [1]:
%load_ext beam_setup

Setting up the Beam environment for interactive use
Standard modules will be automatically imported so you can use them without explicit import
Done importing packages. It took:  3.9 seconds
Beam library is loaded from path: /home/dsi/elads/projects/beamds/notebooks/../src/beam
The Beam version is: 2.3.2


# Use resource to access files and folders

A resource can be a file or folder path on: (1) local filesystem, s3 and s3 compatible filesystem, hdfs and sftp connection.

In [2]:
path = resource('/tmp/path/to/folder')

The path resource follows the pathlib api

In [3]:
path.mkdir()

In [4]:
file_path = path.joinpath('a.pt')

but it also supports read and write operations

In [5]:
file_path.write(torch.randn(4))

file:///tmp/path/to/folder/a.pt

In [6]:
print(file_path.read())

tensor([ 1.1119, -1.1612,  1.7486,  2.4973])


it supports many file types including: torch (.pt), pickle (.pkl), feather (.fea), parquet (.parquet) and many more (see beam.path.core.PureBeamPath read and write operations)

we can also specify how we would like to store the file

In [7]:
path.joinpath('some_name_with_no_extention').write(np.random.randn(4), ext='.pkl')

file:///tmp/path/to/folder/some_name_with_no_extention

In [8]:
path.joinpath('some_name_with_no_extention').read(ext='.pkl')

array([-0.12440085, -0.25151748, -0.99421076,  0.03682782])

we can also iter and list folders

In [9]:
list(path.iterdir())
list(path)

[file:///tmp/path/to/folder/a.pt,
 file:///tmp/path/to/folder/some_name_with_no_extention]

To access paths on different storage platforms use the standard URI conventions:

path = resource('scheme://\<hostname\>:\<port\>/path/to/my/file?arg1=val1&arg2=val2')

Some examples:
* s3 on AWS: s3:///\<bucket name\>/<\object\>?access_key=\<my aws access key\>&secret_key=\<my secret access key\>
* s3 on Minio: s3://\<hostname\>:\<port\>/\<bucket name\>/<\object\>?access_key=\<my aws access key\>&secret_key=\<my secret access key\>?tls=false
* HDFS: hdfs://\<hostname\>:\<http connection port usually 9870\>/path/to/my/file?access_key=\<my hdfs access key\>?tls=\<whether connecting via https\>


Note that you can replace the scheme s3 with s3-pa and hdfs with hdfs-pa to get access via pyarrow which can increase performance instead of native implementations like boto3.
For hdfs-pa you may need to replace the port to the data node communication port: usually at 50010 (consult hdfs-site.xml and core-site.xml files for details)  

# Use resource to access Large Language Models (LLMs)

You can use resource also to access LLMs on various platforms: openai, fastchat, tgi, internal fastapi, local huggingface

before accessing to the LLM, note that we can store access keys to our environment and to a local file s.t. it stays permanenty in our system

In [57]:
# this would print the stored key in your system
# beam_key['OPENAI_API_KEY']

In [11]:
# this will assign a new key to your system
# beam_key['OPENAI_API_KEY'] = <my new key>

In [13]:
llm = resource('openai:///gpt-4')

you can give any model instructions or chat with it

In [14]:
llm.ask('when israel was founded? give me exact date').text

'Israel was founded on May 14, 1948.'

you can chat with the model

In [15]:
llm.chat('Hi my name is elad').text

'Hello Elad! How can I assist you today?'

In [16]:
llm.chat('Hi again, do you remember my name?').text

'Yes, your name is Elad. How can I assist you further?'

In [19]:
llm.chat('Hi again, do you remember my name?', reset_chat=True).text

"As an AI, I don't have the ability to remember personal data unless it's shared in the current conversation. I am designed to respect user privacy and confidentiality."

You can also parse the response in other formats

In [20]:
llm.ask('Hi how are you today? answer in a JSON format').json

{'status': "I'm an AI and don't have feelings, but I'm functioning as expected",
 'message': 'Ready to assist you'}

In [22]:
llm.ask('Hi how are you today? answer in a YAML format').yaml

{'status': {'message': "I'm an AI, I don't have feelings, but I'm functioning as expected.",
  'code': 200}}

You can use the LLM also directly with langchain without any further wrapper

In [23]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

In [24]:
llm.invoke(text)

'1. "Rainbow Steps"\n2. "ColorStride"\n3. "Vibrant Toes"\n4. "Spectrum Socks"\n5. "Kaleidoscope Kicks"\n6. "Technicolor Treads"\n7. "Prismatic Peds"\n8. "Colorful Comfort"\n9. "Spectrum Steps"\n10. "Vivid Footprints"\n11. "ColorPop Socks"\n12. "Rainbow Walk"\n13. "Vibrant Walk"\n14. "Hue Crew"\n15. "Colorful Cozies"\n16. "Bright Steps"\n17. "Kaleidosocks"\n18. "Colorful Cushions"\n19. "Spectrum Soles"\n20. "Rainbow Wraps"'

In [25]:
llm.invoke(messages)

'"Rainbow Steps"'

With our URIs, You can also use openai syntax with any model (not just openai), simply import our simulator instead of openai

In [8]:
from beam.llm import openai_simulator as openai

In [3]:
llm = resource('openai:///gpt-4')

In [9]:
openai.Completion.create(prompt='2**10=?', model='openai:///text-davinci-003')

<OpenAIObject text_completion id=cmpl-8LUaFSIYElAhWp8lHG6doTZPl8AFA at 0x7f2424166ca0> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8LUaFSIYElAhWp8lHG6doTZPl8AFA",
  "object": "text_completion",
  "created": 1700133143,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\n1024",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 5,
    "completion_tokens": 3,
    "total_tokens": 8
  }
}

To access other LLM resource types follow the URI convention:

llm = resource('scheme://\<hostname\>:\<port\>/path/to/my/file?arg1=val1&arg2=val2')

possible schemes: openai, fastchat, tgi, fastapi, huggingface

# Use resource to access BeamServer algorithms

You can use beam also to quickly deploy an algorithm via ssh. You can then access this algorithm with a resource object from any machine that can access the server.

For example, here we build a sparse similarity server (like faiss but for sparse vectors e.g. TFIDF vectors)

In [2]:
from beam.sparse import SparseSimilarity
from beam.serve import BeamServer

In [12]:
M = 40000
sparse_sim = SparseSimilarity(metric='cosine', format='coo', vec_size=M, device='cuda', k=10)
server = BeamServer(sparse_sim)
server.run_non_blocking(server='waitress')

2023-11-16 12:06:56 | BeamLog | INFO | Opening a flask inference serve on port: 28951


In [8]:
def gen_coo_vectors(k, M, nel):

    r = []
    c = []
    v = []

    for i in range(k):
        r.append(i * torch.ones(nel, dtype=torch.int64))
        c.append(torch.randint(M, size=(nel,)))
        v.append(torch.randn(nel))

    return torch.sparse_coo_tensor(torch.stack([torch.cat(r), torch.cat(c)]), torch.cat(v), size=(k, M))

In [9]:
s1 = gen_coo_vectors(20000, M, 100)
s2 = gen_coo_vectors(20, M, 100)

In [13]:
sparse_sim = resource('beam-server://localhost:28951')

In [14]:
sparse_sim.add(s1)

In [16]:
%%time
dist, ind = sparse_sim.search(s2, k=10)

CPU times: user 1.58 s, sys: 274 ms, total: 1.86 s
Wall time: 229 ms


The beam server can wrap any function or class for quick and easy deployment

In [49]:
def llm_executor_and_store(prompt, llm='openai:///text-davinci-003?max_tokens=2048'):
    llm = resource(llm)
    code = llm.ask(f"Return executable python code that performs the following task. The final result should assigned to a variable name 'res':\n{prompt}\n\n\n").text
    try:
        exec(code)
        return res, code
    except:
        return 'ExecutionError', code

In [53]:
server = BeamServer(llm_executor_and_store)
server.run_non_blocking(server='waitress')

2023-11-16 12:25:20 | BeamLog | INFO | Opening a flask inference serve on port: 28952


In [54]:
remote_executor = resource('beam-server://localhost:28952')

In [55]:
r, c = remote_executor('what is the 18th number in the fibonacci series?')

print(c)
print(r)

def fibonacci(n): 
    a = 0
    b = 1
    if n < 0: 
        print("Incorrect input") 
    elif n == 0: 
        return a 
    elif n == 1: 
        return b 
    else: 
        for i in range(2,n): 
            c = a + b 
            a = b 
            b = c 
        return b 
  
res = fibonacci(18)
1597
